In [1]:
import time
import pyspark
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import csv

In [ ]:
file_name = 'ratings.dat'
sc = pyspark.SparkContext(appName='hw4-q2&3')
text_file = sc.textFile(file_name)
test = text_file.collect()

In [ ]:
#將資料以"::"分隔
def deal_process(data):
    data = data.map(lambda x : x.split('::'))
    return data

In [ ]:
#Establish matrix - user-user
def cal_matrix_user(UserID_avg , MovieID_avg , Compare_Data):
    matrix = [[None] * len(MovieID_avg) for i in range(len(UserID_avg))]
    for i in range(len(UserID_avg)):
        for j in range(len(MovieID_avg)):
            frequency = 0
            for z in range(len(Compare_Data)):
                if (str(UserID_avg[i][0]) == str(Compare_Data[z][0])) and (str(MovieID_avg[j][0]) == str(Compare_Data[z][1])):
                    frequency = frequency + Compare_Data[z][2] - UserID_avg[i][1]
            matrix[i][j] = frequency
    return matrix

In [ ]:
#Establish matrix - item-item
def cal_matrix_item(UserID_avg , MovieID_avg , Compare_Data):
    matrix = [[None] * len(UserID_avg) for i in range(len(MovieID_avg))]
    for i in range(len(MovieID_avg)):
        for j in range(len(UserID_avg)):
            frequency = 0
            for z in range(len(Compare_Data)):
                if (str(UserID_avg[j][0]) == str(Compare_Data[z][0])) and (str(MovieID_avg[i][0]) == str(Compare_Data[z][1])):
                    frequency = frequency + Compare_Data[z][2] - MovieID_avg[i][1]
            matrix[i][j] = frequency
    return matrix

In [ ]:
#cosine_similarity
def cos_sim(matrix , UserID_number , MovieID_number , tp):
    list = []
    if (tp == 0):
        for i in range (1):
            for j in range(i,UserID_number):
                tmp = []
                vector1 = np.array(matrix[i])
                vector2 = np.array(matrix[j])
                tmp.append('S'+str(i+1)+str(j+1))
                if(i==j):
                    tmp.append(1)
                elif((np.linalg.norm(vector1)*(np.linalg.norm(vector2)!=0))):
                    tmp.append(np.dot(vector1,vector2) / (np.linalg.norm(vector1)*(np.linalg.norm(vector2))))
                else:
                    tmp.append(0)
                list.append(tmp)
    else:
        for i in range (1):
            for j in range(i,MovieID_number):
                tmp = []
                vector1 = np.array(matrix[i])
                vector2 = np.array(matrix[j])
                tmp.append('S'+str(i+1)+str(j+1))
                if(i==j):
                    tmp.append(1)
                elif((np.linalg.norm(vector1)*(np.linalg.norm(vector2)!=0))):
                    tmp.append(np.dot(vector1,vector2) / (np.linalg.norm(vector1)*(np.linalg.norm(vector2))))
                else:
                    tmp.append(0)
                list.append(tmp)
    return list

In [ ]:
data = deal_process(text_file)
spark = SparkSession.builder.getOrCreate()
spark_df = spark.createDataFrame(data).toDF('UserID','MovieID','Rating','Timestamp')
# spark_df.show()

In [ ]:
#Dataframe deal with data
spark_df = spark_df.withColumn('Rating', spark_df['Rating'].cast('double'))
spark_df = spark_df.withColumn('UserID', spark_df['UserID'].cast('int'))
spark_df = spark_df.withColumn('MovieID', spark_df['MovieID'].cast('int'))
# spark_df.printSchema()
avg = spark_df.select('UserID','MovieID','Rating')
#ALL_UserID -> 6040 , Test_UserID -> 5
#UserID_avg(Rating)
UserID_avg = avg.groupBy('UserID').mean().sort('UserID',ascending=True)
#ALL_MovieID -> 3706 , Test_MovieID -> 35
#MovieID_avg(Rating)
MovieID_avg = avg.groupBy('MovieID').mean().sort('MovieID',ascending=True)
#Compare_Data , Test_Compara_data -> 50
Compare_Data = avg.groupBy('UserID','MovieID').mean().sort('UserID',ascending=True)
#drop no-need data,Adjust type
UserID_avg = UserID_avg.withColumn('UserID', UserID_avg['UserID'].cast('String'))
MovieID_avg = MovieID_avg.withColumn('MovieID', MovieID_avg['MovieID'].cast('String'))
Compare_Data = Compare_Data.withColumn('UserID', Compare_Data['UserID'].cast('String'))
Compare_Data = Compare_Data.withColumn('MovieID', Compare_Data['MovieID'].cast('String'))
UserID_avg = UserID_avg.drop('avg(UserID)')
UserID_avg = UserID_avg.drop('avg(MovieID)')
MovieID_avg = MovieID_avg.drop('avg(UserID)')
MovieID_avg = MovieID_avg.drop('avg(MovieID)')
Compare_Data = Compare_Data.drop('avg(UserID)')
Compare_Data = Compare_Data.drop('avg(MovieID)')
User_number = UserID_avg.count()
Movie_number = MovieID_avg.count()
Compare_Data_number = Compare_Data.count()
# print('----------------------------------')
# UserID_avg.printSchema()
# print(UserID_avg.count())
# UserID_avg.show()
# print('----------------------------------')
# MovieID_avg.printSchema()
# print(MovieID_avg.count())
# MovieID_avg.show()
# print('----------------------------------')
# Compare_Data.printSchema()
# Compare_Data.show(100)
# print('----------------------------------')

In [ ]:
#Dataframe transform list
# print('----------------------------------')
# print('Compare_Data_list : ')
Compare_Data = Compare_Data.toPandas()
Compare_Data = np.array(Compare_Data)
Compare_Data = Compare_Data.tolist()
# print('----------------------------------')
# print(Compare_Data)
# print('----------------------------------')
# print('')
# print('----------------------------------')
# print('UserID_avg_list : ')
UserID_avg = UserID_avg.toPandas()
UserID_avg = np.array(UserID_avg)
UserID_avg = UserID_avg.tolist()
# print('----------------------------------')
# print(UserID_avg)
# print('----------------------------------')
# print('')
# print('MovieID_avg_list : ')
# print('----------------------------------')
MovieID_avg = MovieID_avg.toPandas()
MovieID_avg = np.array(MovieID_avg)
MovieID_avg = MovieID_avg.tolist()
# print(MovieID_avg)
# print('----------------------------------')

In [ ]:
#User-user sim
test_user = cal_matrix_user(UserID_avg,MovieID_avg,Compare_Data)
result_cos_sim = cos_sim(test_user , len(UserID_avg) , len(MovieID_avg) , 0)
result_cos_sim.sort(key = lambda x : x[1] , reverse=True)
with open('User_User_Similarity.txt', 'w') as f:
    for item in result_cos_sim:
        f.write("%s\n" % item)

In [ ]:
#item-item sim
test_item = cal_matrix_item(UserID_avg,MovieID_avg,Compare_Data)
result_cos_sim = cos_sim(test_item , len(UserID_avg) , len(MovieID_avg) , 1)
result_cos_sim.sort(key = lambda x : x[1] , reverse=True)
with open('Item_Item_Similarity.txt', 'w') as f:
    for item in result_cos_sim:
        f.write("%s\n" % item)

In [ ]:
sc.stop()